In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install biopython
!brew install blast
# Install BLAST and required bioinformatics tools
!apt-get update
!apt-get install -y ncbi-blast+
!pip install biopython
# Verify installation
!blastp -version
!makeblastdb -version

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 47.8 MB/s eta 0:00:00
/bin/bash: line 1: brew: command not found
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,086 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,288 kB]
Hit:12 https:/

In [25]:
# # Import all required libraries
from Bio.Blast.Applications import NcbiblastpCommandline, NcbiblastxCommandline
from Bio import SeqIO
from Bio.Blast import NCBIWWW
import subprocess
import os
import tempfile
import pandas as pd
from IPython.display import display, HTML
import time
from io import StringIO

In [ ]:
# Step 1: Parse human sequences
path = ''
human_fasta = path+"human.fa"
mouse_fasta = path+"mouse.fa"

human_sequences = list(SeqIO.parse(human_fasta, "fasta"))
print(f"Found {len(human_sequences)} human sequences")

# Display first few sequences
print("\nFirst 3 human sequences:")
for i, seq in enumerate(human_sequences):
    print(f"  {i+1}. {seq.id} - Length: {len(seq.seq)}")

Found 13 human sequences

First 3 human sequences:
  1. STAU1 - Length: 577
  2. NRAS - Length: 189
  3. HRAS - Length: 189
  4. SLC30A10 - Length: 496
  5. CTSV - Length: 334
  6. SFRS4 - Length: 494
  7. GABRG2 - Length: 467
  8. PCDH1 - Length: 1060
  9. CAMK1G - Length: 476
  10. ADCYAP1 - Length: 315
  11. PPP2R5D - Length: 602
  12. MLLT3 - Length: 607
  13. VN1R3 - Length: 311


In [28]:
def determine_sequence_type(sp_sequences):
    try:
        sequences = sp_sequences
        if not sequences:
            return "unknown", "No sequences found"

        first_seq = str(sequences[0].seq)

        clean_seq = first_seq.upper().replace('-', '').replace('*', '').replace('X', '')

        if not clean_seq:  # Empty sequence
            return "unknown", "Empty sequence"

        nucleotide_chars = set('ATCGN')
        protein_chars = set('ACDEFGHIKLMNPQRSTVWY')

        seq_chars = set(clean_seq)

        nucleotide_count = sum(1 for c in clean_seq if c in nucleotide_chars)
        nucleotide_ratio = nucleotide_count / len(clean_seq)

        protein_count = sum(1 for c in clean_seq if c in protein_chars)
        protein_ratio = protein_count / len(clean_seq)

        print(f"  Sequence analysis: {len(clean_seq)} chars, "
              f"nucleotide_ratio: {nucleotide_ratio:.3f}, "
              f"protein_ratio: {protein_ratio:.3f}")

        if nucleotide_ratio > 0.9 and protein_ratio < 0.5:
            return "nucleotide", f"Primarily nucleotide chars ({nucleotide_ratio:.1%})"
        elif protein_ratio > 0.9 and nucleotide_ratio < 0.5:
            return "protein", f"Primarily protein chars ({protein_ratio:.1%})"
        else:
            if any(c in 'EFILPQ' for c in clean_seq):
                return "protein", "Contains amino-acid specific characters"
            elif len(clean_seq) % 3 == 0 and nucleotide_ratio > 0.7:
                return "nucleotide", "Length divisible by 3 and primarily ATCG"
            else:
                return "ambiguous", f"Could not reliably determine (nuc: {nucleotide_ratio:.1%}, prot: {protein_ratio:.1%})"

    except Exception as e:
        return "error", f"Error analyzing: {str(e)}"

def select_blast_program(query_type, db_type):
    """Select the appropriate BLAST program based on query and database types"""
    program_map = {
        ('protein', 'protein'): ('blastp', 'prot'),
        ('nucleotide', 'nucleotide'): ('blastn', 'nucl'),
        ('nucleotide', 'protein'): ('blastx', 'prot'),  # Translates nucleotide query
        ('protein', 'nucleotide'): ('tblastn', 'nucl')  # Translates nucleotide database
    }

    return program_map.get((query_type, db_type), ('blastp', 'prot'))

In [29]:
# Step 3: Create BLAST database
mouse_sequences = list(SeqIO.parse(mouse_fasta, "fasta"))
human_type, human_reason = determine_sequence_type(human_sequences)
mouse_type, mouse_reason = determine_sequence_type(mouse_fasta)

blast_program, db_type = select_blast_program(human_type, mouse_type)
print(f"Selected BLAST program: {blast_program} (database type: {db_type})")

db_name = "mouse_db"

make_db_cmd = [
    "makeblastdb",
    "-in", mouse_fasta,
    "-dbtype", db_type,
    "-out", db_name,
    "-parse_seqids"
]

result = subprocess.run(make_db_cmd, capture_output=True, text=True)

if result.returncode == 0:
    print(" BLAST database created successfully")
else:
    print(f" Error creating database: {result.stderr}")
    # Check if database files exist
    db_files = [f"{db_name}.{ext}" for ext in (["nhr", "nin", "nog"] if db_type == "nucl" else ["phr", "pin", "pog"])]
    existing_files = [f for f in db_files if os.path.exists(f)]
    if existing_files:
        print(f" Using existing database files")
    else:
        print(" Cannot proceed without database")

  Sequence analysis: 577 chars, nucleotide_ratio: 0.260, protein_ratio: 1.000
Selected BLAST program: blastp (database type: prot)
 BLAST database created successfully


In [30]:
result

CompletedProcess(args=['makeblastdb', '-in', '/content/drive/MyDrive/Biomedical_DS/homework3/mouse.fa', '-dbtype', 'prot', '-out', 'mouse_db', '-parse_seqids'], returncode=0, stdout='\n\nBuilding a new DB, current time: 10/28/2025 02:51:39\nNew DB name:   /content/mouse_db\nNew DB title:  /content/drive/MyDrive/Biomedical_DS/homework3/mouse.fa\nSequence type: Protein\nDeleted existing Protein BLAST database named /content/mouse_db\nKeep MBits: T\nMaximum file size: 1000000000B\nAdding sequences from FASTA; added 17489 sequences in 0.88258 seconds.\n\n\n', stderr='')

In [31]:
# Step 4: Run BLAST for each human sequence
print(f"\n Running BLAST searches...")

results_data = []

for i, human_seq in enumerate(human_sequences):
    print(f"  Processing {i+1}/{len(human_sequences)}: {human_seq.id}")

    # Create temporary file for query sequence
    with tempfile.NamedTemporaryFile(mode='w', suffix='.fasta', delete=False) as temp_file:
        SeqIO.write(human_seq, temp_file.name, "fasta")
        temp_path = temp_file.name

    try:
        # Choose BLAST program based on database type
        if db_type == "prot":
            blast_cline = NcbiblastpCommandline(
                query=temp_path,
                db=db_name,
                evalue=0.001,
                outfmt="6 qseqid sseqid evalue bitscore length pident qcovs",
                max_target_seqs=1,
                matrix="BLOSUM80",
                gapopen=11,
                gapextend=1
            )
        else:
            blast_cline = NcbiblastxCommandline(
                query=temp_path,
                db=db_name,
                evalue=0.001,
                outfmt="6 qseqid sseqid evalue bitscore length pident qcovs",
                max_target_seqs=1,
                matrix="BLOSUM80",
                gapopen=11,
                gapextend=1
            )

        stdout, stderr = blast_cline()

        if stderr:
            print(f" Warning: {stderr.strip()}")

        if stdout.strip():
            lines = stdout.strip().split('\n')
            for line in lines:
                if line and not line.startswith("#"):
                    fields = line.split('\t')
                    if len(fields) >= 7:
                        result_entry = {
                            'Human_ID': human_seq.id,
                            'Mouse_Homolog_ID': fields[1],
                            'E-value': float(fields[2]),
                            'Bitscore': float(fields[3]),
                            'Alignment_Length': int(fields[4]),
                            'Percent_Identity': float(fields[5]),
                            'Query_Coverage': float(fields[6])
                        }
                        results_data.append(result_entry)
                        print(f" Found: {fields[1]} (E-value: {fields[2]}, Identity: {fields[5]}%)")
        else:
            print(f" No significant hits found")

    except Exception as e:
        print(f" Error: {str(e)}")
    finally:
        # Clean up temporary file
        if os.path.exists(temp_path):
            os.unlink(temp_path)



 Running BLAST searches...
  Processing 1/13: STAU1
 Found: sp|Q9Z108|STAU1_MOUSE (E-value: 0.0, Identity: 91.207%)
  Processing 2/13: NRAS
 Found: sp|P08556|RASN_MOUSE (E-value: 2.81e-144, Identity: 98.413%)
  Processing 3/13: HRAS
 Found: sp|Q61411|RASH_MOUSE (E-value: 1.15e-146, Identity: 100.000%)
  Processing 4/13: SLC30A10
 Found: sp|Q3UVU3|ZNT10_MOUSE (E-value: 0.0, Identity: 76.210%)
  Processing 5/13: CTSV
 Found: sp|P06797|CATL1_MOUSE (E-value: 0.0, Identity: 75.449%)
  Processing 6/13: SFRS4
 Found: sp|Q8VE97|SRSF4_MOUSE (E-value: 3.02e-153, Identity: 94.737%)
  Processing 7/13: GABRG2
 Found: sp|P22723|GBRG2_MOUSE (E-value: 0.0, Identity: 97.474%)
  Processing 8/13: PCDH1
 Found: sp|Q8BIZ0|PCD20_MOUSE (E-value: 6.74e-128, Identity: 34.977%)
  Processing 9/13: CAMK1G
 Found: sp|Q91VB2|KCC1G_MOUSE (E-value: 0.0, Identity: 92.034%)
  Processing 10/13: ADCYAP1
 Found: sp|O70176|PACA_MOUSE (E-value: 5.74e-97, Identity: 82.386%)
  Processing 11/13: PPP2R5D
 Found: sp|Q60996|2A5G

In [32]:
# Step 5: Save results
output_file=path+"blast_results.csv"
if results_data:
    df = pd.DataFrame(results_data)
    df.to_csv(output_file, sep='\t', index=False)
    print(f"\n Results saved to: {output_file}")
    print(f" Total homolog pairs found: {len(results_data)}")

    print(f"\n Summary Statistics:")
    print(f"   Average E-value: {df['E-value'].mean():.2e}")
    print(f"   Average Identity: {df['Percent_Identity'].mean():.2f}%")
    print(f"   Best Identity: {df['Percent_Identity'].max():.2f}%")
    print(f"   Best E-value: {df['E-value'].min():.2e}")
else:
    print(" No homologs found in the analysis")


 Results saved to: /content/drive/MyDrive/Biomedical_DS/homework3/blast_results.csv
 Total homolog pairs found: 13

 Summary Statistics:
   Average E-value: 5.59e-25
   Average Identity: 81.24%
   Best Identity: 100.00%
   Best E-value: 0.00e+00


# Parameter explanation
BLAST PARAMETER CHOICES EXPLANATION

Sequence type detection:
• I Checked both human (query) and mouse (database) sequences
• Automatically selects the correct BLAST program based on percentage of presence of nucleotide and protien character

Here is the logic behind choosing BLAST algorithm:   
• blastp:    Protein query vs Protein database   
• blastn:    Nucleotide query vs Nucleotide database.  
• blastx:    Nucleotide query vs Protein database (translates query in all frames)  
• tblastn:   Protein query vs Nucleotide database (translates database in all frames)


(i) **BLAST Program**: Auto-selected based on sequence types.
    In our case both query and database sequences were protien, so I chose blastp algorithm.

(ii) **Substitution Matrix:**   
    Out of BLOSUM and PAM, I choose BLOSUM as:  
    • Protein comparisons: BLOSUM62 (standard for evolutionary relationships), initially tested with this and got an average identity of 80.45%.
  ##### As BLOSUM matrices are numbered by the percent identity threshold used:
    BLOSUM30 = sequences with ≤30% identity  # Very distant relationships
    BLOSUM45 = sequences with ≤45% identity  # Distant relationships  
    BLOSUM62 = sequences with ≤62% identity  # Moderate relationships
    BLOSUM80 = sequences with ≤80% identity  # Close relationships
    BLOSUM90 = sequences with ≤90% identity  # Very close relationships

##### Why BLOSUM80?
#####• As shown in the summary statistics Human and mouse share ~80% protein identity, looking at this high similarity score, I switched to BLOSUM80 from BLOSUM62 which improved the similary score from 80.45% to 81.24.   
##### • BLOSUM80 is ideal for detecting high evolutionary relationships

(iii) **Key Parameters:  **
    • E-value threshold: 0.001 - Standard for significant homology. 0.001 means we expect 0.001 random matches by chance (1 in 1000). This also balances sensitivity (finding true homologs) vs specificity (avoiding false positives)  
    • max_target_seqs: 1 - Top homolog only as requested. We only need the single best match for each human sequence. Improves efficiency and reduces output size.  
    • Gap penalties: Appropriate for sequence type (protein vs nucleotide). Higher gap opening penalty gapopen=11 discourages introducing gaps. Lower gap extension penalty gapextend=1 allows existing gaps to extend slightly.